We initialize the camera and the car

In [1]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_fps=40)

from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

We load the road following model

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model2-Copy1.pth'))

<All keys matched successfully>

We set up the car and load the preprocess of images

In [3]:
from utils import preprocess

STEERING_GAIN = -2
STEERING_BIAS = -0.3

We load the signal recognition model

In [4]:
import torchvision
from torch2trt import TRTModule

#model = TRTModule()
#ALEXNET
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 8)
model.load_state_dict(torch.load('recognition_model.pth'))
device = torch.device('cuda')
model = model.to(device)

We set up the display

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
import time

#image = widgets.Image(format='jpeg', width=224, height=224)
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

blocked_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Blocked probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

free_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Free probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)


fid1_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid1 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid2_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid2 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)
fid3_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid3 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid4_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid4 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid5_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid5 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid6_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid6 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

steering_disp = widgets.FloatProgress(
    value=0.0,
    min=-1.0,
    max=1.0,
    description='Direction:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fps = widgets.FloatText(
    value=0.0,
    description='FPS:',
    disabled=False
)

obj = widgets.FloatText(
    value=1,
    description='Objetivo:',
    disabled=False
)



Main function

In [ ]:
display(blocked_disp,free_disp,fid1_disp,fid2_disp,fid3_disp,fid4_disp,fid5_disp,fid6_disp,steering_disp,fps,obj)
import torch.nn.functional as F
import time

t_dir = 0.0
n_dir = 0.0
t_tot = 0.0
n_tot = 0
fps_tot = 0
t_tot_1 = time.time() 

objetivo = 1
obj.value = objetivo
turn_left_start_time_fid1 = None 
turn_right_start_time_fid4 = None 
turn_duration = 1
while True:
    t_tot = time.time()
    fps_tot = 1/(t_tot-t_tot_1)
    t_tot_1 = t_tot
    fps.value = fps_tot
    for i in range(3):
        x = camera.read()
    x = preprocess(x)
    y = model(x)
    

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    blocked_disp.value = prob_blocked
    prob_fid1 = float(y.flatten()[1])
    fid1_disp.value = prob_fid1
    prob_fid2 = float(y.flatten()[2])
    fid2_disp.value = prob_fid2
    prob_fid3 = float(y.flatten()[3])
    fid3_disp.value = prob_fid3
    prob_fid4 = float(y.flatten()[4])
    fid4_disp.value = prob_fid4
    prob_fid5 = float(y.flatten()[5])
    fid5_disp.value = prob_fid5
    prob_fid6 = float(y.flatten()[6])
    fid6_disp.value = prob_fid6
    prob_free = float(y.flatten()[7])
    free_disp.value = prob_free
    
    
    prob_max = float(max(y.flatten()))
    dirrect = "left"
    if prob_blocked == prob_max:
        car.throttle = 0.0
    elif prob_fid1 == prob_max and objetivo == 1:
        turn_left_start_time_fid1 = time.time()
        if dirrect == "right":
            while (time.time() - turn_left_start_time_fid1 < turn_duration ):
                car.throttle = -0.35
                car.steering = -1
        elif dirrect == "left":
            while (time.time() - turn_left_start_time_fid1 < turn_duration ):
                car.throttle = -0.35
                car.steering = 1
    elif prob_fid4 == prob_max and objetivo == 1:
        turn_right_start_time_fid4 = time.time()
    elif prob_fid6 == prob_max and objetivo == 6:
        objetivo = 1
        car.throttle = 0.0
        time.sleep(6)
        obj.value = objetivo
    else:
        time_stop = 0
        car.throttle = -0.35
        for i in range(3):
            x = camera.read()
        x = preprocess(x)
        x = x.half()
        output = model_trt(x).detach().cpu().numpy().flatten()
        x1 = float(output[0])
        car.steering = x1 * STEERING_GAIN + STEERING_BIAS
        steering_disp.value = x1



FloatProgress(value=0.5, bar_style='info', description='Blocked probability:', max=1.0, style=ProgressStyle(ba…

FloatProgress(value=0.5, bar_style='info', description='Free probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid1 probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid2 probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid3 probability:', max=1.0, style=ProgressStyle(bar_c…

...

FloatProgress(value=0.5, bar_style='info', description='Fid5 probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid6 probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.0, bar_style='info', description='Direction:', max=1.0, min=-1.0, style=ProgressStyle(ba…

FloatText(value=0.0, description='FPS:')

FloatText(value=1.0, description='Objetivo:')

Stop the car

In [2]:
car.throttle = 0.0

NameError: name 'camera' is not defined

In [ ]:
camera.running = False
camera.cap.release()